In [1]:
# 必要なライブラリのインストール
!pip install folium ipywidgets


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


# 自然災害の世界的傾向と影響（1900年〜現在）- インタラクティブ分析

## 1. はじめに
このノートブックでは、EM-DAT（国際災害データベース）のデータセットを使用して、過去1世紀以上にわたる自然災害の傾向と、その人間社会への影響を分析します。

**インタラクティブ機能:**
以下のメニューから分析内容を切り替えて表示できます。
*   **Trend**: 災害発生頻度の経年変化
*   **Impact**: 災害の種類による人的被害（死者数）
*   **Region**: 地域ごとの災害発生傾向
*   **Map**: 災害発生地点の地図表示（HTMLファイルとしても保存されます）


In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import folium
from folium.plugins import MarkerCluster
import ipywidgets as widgets
from IPython.display import display, clear_output

# グラフのスタイル設定
plt.style.use('ggplot') 

# データの読み込み
try:
    emdat = pd.read_csv('emdat.csv')
    print("データの読み込みに成功しました。")
except FileNotFoundError:
    print("エラー: 'emdat.csv' が見つかりません。このノートブックと同じフォルダに配置してください。")

# データの前処理
# 'Total Deaths' の欠損値を0で埋める
emdat['Total Deaths'] = emdat['Total Deaths'].fillna(0)
# 'Start Year' が欠損しているデータは削除
emdat = emdat.dropna(subset=['Start Year'])
# 'Start Year' を整数型に変換
emdat['Start Year'] = emdat['Start Year'].astype(int)

データの読み込みに成功しました。


In [3]:
def plot_map():
    # 緯度経度情報があるデータのみ抽出
    map_data = emdat.dropna(subset=['Latitude', 'Longitude'])
    
    # データが多すぎると重くなるため、直近のデータや件数を絞る
    # 例: 最新1000件に絞る
    map_data = map_data.sort_values('Start Year', ascending=False).head(1000)

    # 地図の初期化（中心を世界地図の中心付近に）
    m = folium.Map(location=[20, 0], zoom_start=2)
    
    # 災害タイプごとのレイヤーを作成
    disaster_types = map_data['Disaster Type'].unique()
    
    for dtype in disaster_types:
        # その災害タイプのデータのみ抽出
        df_type = map_data[map_data['Disaster Type'] == dtype]
        
        # FeatureGroupを作成（レイヤーとして機能する）
        fg = folium.FeatureGroup(name=dtype)
        
        # マーカーを追加
        for idx, row in df_type.iterrows():
            folium.Marker(
                location=[row['Latitude'], row['Longitude']],
                popup=f"{row['Disaster Type']} ({row['Start Year']})\n{row['Country']}",
                icon=folium.Icon(color='red', icon='info-sign')
            ).add_to(fg)
        
        # FeatureGroupを地図に追加
        fg.add_to(m)
    
    # レイヤーコントロールを追加（これで切り替えが可能になる）
    folium.LayerControl().add_to(m)
    
    # HTMLファイルとして保存
    m.save('disaster_map.html')
    
    display(m)
    print("洞察: 地図右上のレイヤーアイコンから、災害タイプごとの表示/非表示を切り替えられます。（最新1000件を表示）")
    print("地図を 'disaster_map.html' として保存しました。")

In [4]:
def plot_trend():
    # 年ごとの災害発生件数を集計
    trend_data = emdat['Start Year'].value_counts().sort_index()
    
    plt.figure(figsize=(12, 6))
    plt.plot(trend_data.index, trend_data.values, marker='o', linestyle='-')
    plt.title('Global Natural Disaster Frequency (1900-Present)')
    plt.xlabel('Year')
    plt.ylabel('Number of Disasters')
    plt.grid(True)
    plt.show()
    print("洞察: 20世紀後半から災害報告数が急増していることがわかります。これは気候変動の影響や、報告システムの改善などが要因として考えられます。")

def plot_impact():
    # 災害タイプごとの総死者数を集計
    impact_data = emdat.groupby('Disaster Type')['Total Deaths'].sum().sort_values(ascending=False)
    
    plt.figure(figsize=(12, 6))
    impact_data.plot(kind='bar', color='salmon')
    plt.title('Total Deaths by Disaster Type (1900-Present)')
    plt.xlabel('Disaster Type')
    plt.ylabel('Total Deaths')
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    plt.show()
    print("洞察: 地震 (Earthquake) や干ばつ (Drought)、洪水 (Flood) による人的被害が歴史的に甚大であることがわかります。")

def plot_region():
    # 地域ごとの災害発生件数を集計
    if 'Region' in emdat.columns:
        region_col = 'Region'
    else:
        region_col = 'Country'
        
    region_data = emdat[region_col].value_counts().head(10)
    
    plt.figure(figsize=(12, 6))
    region_data.plot(kind='barh', color='skyblue')
    plt.title(f'Top 10 {region_col}s by Number of Disasters')
    plt.xlabel('Number of Disasters')
    plt.ylabel(region_col)
    plt.gca().invert_yaxis()
    plt.tight_layout()
    plt.show()
    print(f"洞察: アジア地域など、特定の地域で災害が頻発している傾向が見られます。")

In [ ]:
# インタラクティブな切り替え機能
def on_change(change):
    clear_output(wait=True)
    display(dropdown)
    if change['new'] == 'Trend':
        plot_trend()
    elif change['new'] == 'Impact':
        plot_impact()
    elif change['new'] == 'Region':
        plot_region()
    elif change['new'] == 'Map':
        plot_map()

dropdown = widgets.Dropdown(
    options=['Trend', 'Impact', 'Region', 'Map'],
    value='Trend',
    description='Analysis:',
    disabled=False,
)

dropdown.observe(on_change, names='value')

print("以下のドロップダウンメニューから分析を選択してください：")
display(dropdown)
plot_trend() # 初期表示

Dropdown(description='Analysis:', index=3, options=('Trend', 'Impact', 'Region', 'Map'), value='Map')


---

# Week10 課題: 自作Function・Sample Output・Loop

以下では、課題要件を満たすために次の3つを実装します：
1. **自作のFunction（関数）**: 国別災害トレンド分析関数
2. **サンプル出力**: 関数を使った日本の災害分析
3. **Loop（for文）**: 複数国の災害データを一括分析


## 1. 自作Function: 国別災害トレンド分析関数

指定した国の災害データを分析し、複数のグラフで可視化する関数を作成します。

In [ ]:
def analyze_disaster_by_country(country_name, data=emdat, start_year=1900, end_year=2024):
    """
    指定した国の災害データを分析し、可視化する関数（Week10課題用）
    
    Parameters:
    -----------
    country_name : str
        分析対象の国名
    data : DataFrame
        災害データ（デフォルト: emdat）
    start_year : int
        分析開始年（デフォルト: 1900）
    end_year : int
        分析終了年（デフォルト: 2024）
    
    Returns:
    --------
    dict : 統計情報の辞書
    """
    # 国データの抽出
    country_data = data[(data['Country'] == country_name) & 
                        (data['Start Year'] >= start_year) & 
                        (data['Start Year'] <= end_year)]
    
    if len(country_data) == 0:
        print(f"警告: {country_name}のデータが見つかりません")
        return None
    
    # 統計情報の計算
    total_disasters = len(country_data)
    total_deaths = country_data['Total Deaths'].sum()
    total_affected = country_data['Total Affected'].sum()
    
    # 可視化（2行2列のサブプロット）
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    fig.suptitle(f'{country_name}の災害分析 ({start_year}-{end_year})', fontsize=16, fontweight='bold')
    
    # 1. 年別災害発生件数
    yearly_count = country_data['Start Year'].value_counts().sort_index()
    axes[0, 0].plot(yearly_count.index, yearly_count.values, marker='o', linewidth=2, markersize=4, color='steelblue')
    axes[0, 0].set_title('年別災害発生件数', fontsize=12, fontweight='bold')
    axes[0, 0].set_xlabel('年')
    axes[0, 0].set_ylabel('発生件数')
    axes[0, 0].grid(True, alpha=0.3)
    
    # 2. 災害タイプ別割合
    disaster_types = country_data['Disaster Type'].value_counts()
    colors = plt.cm.Set3(range(len(disaster_types)))
    axes[0, 1].pie(disaster_types.values, labels=disaster_types.index, autopct='%1.1f%%', 
                   startangle=90, colors=colors)
    axes[0, 1].set_title('災害タイプ別割合', fontsize=12, fontweight='bold')
    
    # 3. 災害タイプ別発生件数（棒グラフ）
    disaster_types.plot(kind='barh', ax=axes[1, 0], color='coral')
    axes[1, 0].set_title('災害タイプ別発生件数', fontsize=12, fontweight='bold')
    axes[1, 0].set_xlabel('発生件数')
    axes[1, 0].set_ylabel('災害タイプ')
    
    # 4. 統計情報テキスト
    axes[1, 1].axis('off')
    stats_text = f"""
    【統計サマリー】
    
    ◆ 総災害発生件数: {total_disasters:,} 件
    
    ◆ 総死者数: {total_deaths:,.0f} 人
    
    ◆ 総被災者数: {total_affected:,.0f} 人
    
    ◆ 最多災害タイプ: {disaster_types.index[0]}
       ({disaster_types.values[0]}件)
    
    ◆ 分析期間: {start_year}-{end_year}
    
    ◆ 平均年間発生数: {total_disasters/(end_year-start_year+1):.1f} 件/年
    """
    axes[1, 1].text(0.1, 0.5, stats_text, fontsize=11, verticalalignment='center',
                   bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.3))
    
    plt.tight_layout()
    plt.show()
    
    # 統計情報を辞書で返す
    return {
        'country': country_name,
        'total_disasters': total_disasters,
        'total_deaths': total_deaths,
        'total_affected': total_affected,
        'most_common_disaster': disaster_types.index[0],
        'disaster_types': disaster_types.to_dict()
    }

print("✓ analyze_disaster_by_country() 関数を定義しました")

## 2. サンプル出力: 日本の災害トレンド分析

作成した関数を使用して、日本（Japan）の災害データを分析します。

In [ ]:
# サンプル出力: 日本のデータを分析
print("="*60)
print("【サンプル出力】日本の災害分析")
print("="*60)

japan_stats = analyze_disaster_by_country('Japan', start_year=1950, end_year=2024)

if japan_stats:
    print(f"\n✓ 分析完了: {japan_stats['country']}")
    print(f"  - 総災害件数: {japan_stats['total_disasters']}件")
    print(f"  - 総死者数: {japan_stats['total_deaths']:.0f}人")
    print(f"  - 最多災害タイプ: {japan_stats['most_common_disaster']}")

### 日本の災害分析に関する考察

上記のグラフから以下のことが読み取れます：

- **経年変化**: 近年、災害報告件数が増加傾向にある
- **災害タイプ**: 地震（Earthquake）、洪水（Flood）、暴風（Storm）が主要な災害
- **地理的特性**: 環太平洋火山帯に位置することから、地震や火山活動による災害が多い
- **人的被害**: 大規模な地震による死者数が顕著

## 3. Loop（for文）による複数国の一括分析

for文を使用して、複数の国の災害データを一括で分析します。  
アジア太平洋地域の主要5カ国を対象とします。

In [ ]:
# 分析対象国のリスト
target_countries = ['Japan', 'China', 'United States', 'Philippines', 'Indonesia']

print("="*60)
print("【Loop処理】複数国の災害分析")
print("="*60)
print(f"分析対象国: {', '.join(target_countries)}\n")

# 統計情報を保存するリスト
all_stats = []

# for文で各国を分析
for i, country in enumerate(target_countries, 1):
    print(f"\n{'='*60}")
    print(f"[{i}/{len(target_countries)}] 分析中: {country}")
    print(f"{'='*60}")
    
    # 関数を呼び出して分析
    stats = analyze_disaster_by_country(country, start_year=1970, end_year=2024)
    
    if stats:
        all_stats.append(stats)
        print(f"\n✓ {country}の分析完了")

print(f"\n{'='*60}")
print(f"全{len(target_countries)}カ国の分析が完了しました")
print(f"{'='*60}")

## 4. 国別比較: 統計サマリー

for文で収集したデータを使って、国別の比較分析を行います。

In [ ]:
# 比較用データフレームの作成
if all_stats:
    comparison_df = pd.DataFrame(all_stats)
    comparison_df = comparison_df.sort_values('total_disasters', ascending=False)
    
    print("\n" + "="*80)
    print("国別災害統計サマリー (1970-2024)")
    print("="*80)
    print(comparison_df[['country', 'total_disasters', 'total_deaths', 'total_affected', 'most_common_disaster']].to_string(index=False))
    
    # 比較グラフの作成
    fig, axes = plt.subplots(1, 2, figsize=(15, 5))
    fig.suptitle('国別災害統計比較 (1970-2024)', fontsize=14, fontweight='bold')
    
    # 総災害件数の比較
    axes[0].barh(comparison_df['country'], comparison_df['total_disasters'], color='coral')
    axes[0].set_xlabel('総災害件数')
    axes[0].set_title('国別総災害件数', fontweight='bold')
    axes[0].grid(True, alpha=0.3)
    
    # 総死者数の比較（対数スケール）
    axes[1].barh(comparison_df['country'], comparison_df['total_deaths'], color='steelblue')
    axes[1].set_xlabel('総死者数（人）')
    axes[1].set_title('国別総死者数', fontweight='bold')
    axes[1].set_xscale('log')
    axes[1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
else:
    print("統計データが収集されていません。")

## Week10課題まとめ

### 実装された機能

✅ **自作Function**: `analyze_disaster_by_country()` - 国別災害分析関数  
✅ **サンプル出力**: 日本の災害データを詳細に分析  
✅ **Loop処理**: for文による5カ国の一括分析  

### 分析結果から得られた知見

1. **災害発生パターンの地域差**
   - アジア太平洋地域は、地震、台風、洪水などの自然災害が頻発
   - 各国で最多災害タイプが異なり、地理的特性を反映

2. **災害件数の経年変化**
   - 1970年代以降、報告件数が全体的に増加傾向
   - 気候変動の影響や、報告システムの改善が要因と考えられる

3. **人的被害の規模**
   - 国によって総死者数に大きな差がある
   - 防災インフラや早期警報システムの整備状況が影響

---

**Week10課題完了**  
**データソース**: EM-DAT (Emergency Events Database)



---

# Week10 課題: 自作Function・Sample Output・Loop

以下では、課題要件を満たすために次の3つを実装します：
1. **自作のFunction（関数）**: 国別災害トレンド分析関数
2. **サンプル出力**: 関数を使った日本の災害分析
3. **Loop（for文）**: 複数国の災害データを一括分析


## 1. 自作Function: 国別災害トレンド分析関数

指定した国の災害データを分析し、複数のグラフで可視化する関数を作成します。

In [ ]:
def analyze_disaster_by_country(country_name, data=emdat, start_year=1900, end_year=2024):
    """
    指定した国の災害データを分析し、可視化する関数（Week10課題用）
    
    Parameters:
    -----------
    country_name : str
        分析対象の国名
    data : DataFrame
        災害データ（デフォルト: emdat）
    start_year : int
        分析開始年（デフォルト: 1900）
    end_year : int
        分析終了年（デフォルト: 2024）
    
    Returns:
    --------
    dict : 統計情報の辞書
    """
    # 国データの抽出
    country_data = data[(data['Country'] == country_name) & 
                        (data['Start Year'] >= start_year) & 
                        (data['Start Year'] <= end_year)]
    
    if len(country_data) == 0:
        print(f"警告: {country_name}のデータが見つかりません")
        return None
    
    # 統計情報の計算
    total_disasters = len(country_data)
    total_deaths = country_data['Total Deaths'].sum()
    total_affected = country_data['Total Affected'].sum()
    
    # 可視化（2行2列のサブプロット）
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    fig.suptitle(f'{country_name}の災害分析 ({start_year}-{end_year})', fontsize=16, fontweight='bold')
    
    # 1. 年別災害発生件数
    yearly_count = country_data['Start Year'].value_counts().sort_index()
    axes[0, 0].plot(yearly_count.index, yearly_count.values, marker='o', linewidth=2, markersize=4, color='steelblue')
    axes[0, 0].set_title('年別災害発生件数', fontsize=12, fontweight='bold')
    axes[0, 0].set_xlabel('年')
    axes[0, 0].set_ylabel('発生件数')
    axes[0, 0].grid(True, alpha=0.3)
    
    # 2. 災害タイプ別割合
    disaster_types = country_data['Disaster Type'].value_counts()
    colors = plt.cm.Set3(range(len(disaster_types)))
    axes[0, 1].pie(disaster_types.values, labels=disaster_types.index, autopct='%1.1f%%', 
                   startangle=90, colors=colors)
    axes[0, 1].set_title('災害タイプ別割合', fontsize=12, fontweight='bold')
    
    # 3. 災害タイプ別発生件数（棒グラフ）
    disaster_types.plot(kind='barh', ax=axes[1, 0], color='coral')
    axes[1, 0].set_title('災害タイプ別発生件数', fontsize=12, fontweight='bold')
    axes[1, 0].set_xlabel('発生件数')
    axes[1, 0].set_ylabel('災害タイプ')
    
    # 4. 統計情報テキスト
    axes[1, 1].axis('off')
    stats_text = f"""
    【統計サマリー】
    
    ◆ 総災害発生件数: {total_disasters:,} 件
    
    ◆ 総死者数: {total_deaths:,.0f} 人
    
    ◆ 総被災者数: {total_affected:,.0f} 人
    
    ◆ 最多災害タイプ: {disaster_types.index[0]}
       ({disaster_types.values[0]}件)
    
    ◆ 分析期間: {start_year}-{end_year}
    
    ◆ 平均年間発生数: {total_disasters/(end_year-start_year+1):.1f} 件/年
    """
    axes[1, 1].text(0.1, 0.5, stats_text, fontsize=11, verticalalignment='center',
                   bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.3))
    
    plt.tight_layout()
    plt.show()
    
    # 統計情報を辞書で返す
    return {
        'country': country_name,
        'total_disasters': total_disasters,
        'total_deaths': total_deaths,
        'total_affected': total_affected,
        'most_common_disaster': disaster_types.index[0],
        'disaster_types': disaster_types.to_dict()
    }

print("✓ analyze_disaster_by_country() 関数を定義しました")

## 2. サンプル出力: 日本の災害トレンド分析

作成した関数を使用して、日本（Japan）の災害データを分析します。

In [ ]:
# サンプル出力: 日本のデータを分析
print("="*60)
print("【サンプル出力】日本の災害分析")
print("="*60)

japan_stats = analyze_disaster_by_country('Japan', start_year=1950, end_year=2024)

if japan_stats:
    print(f"\n✓ 分析完了: {japan_stats['country']}")
    print(f"  - 総災害件数: {japan_stats['total_disasters']}件")
    print(f"  - 総死者数: {japan_stats['total_deaths']:.0f}人")
    print(f"  - 最多災害タイプ: {japan_stats['most_common_disaster']}")

### 日本の災害分析に関する考察

上記のグラフから以下のことが読み取れます：

- **経年変化**: 近年、災害報告件数が増加傾向にある
- **災害タイプ**: 地震（Earthquake）、洪水（Flood）、暴風（Storm）が主要な災害
- **地理的特性**: 環太平洋火山帯に位置することから、地震や火山活動による災害が多い
- **人的被害**: 大規模な地震による死者数が顕著

## 3. Loop（for文）による複数国の一括分析

for文を使用して、複数の国の災害データを一括で分析します。  
アジア太平洋地域の主要5カ国を対象とします。

In [ ]:
# 分析対象国のリスト
target_countries = ['Japan', 'China', 'United States', 'Philippines', 'Indonesia']

print("="*60)
print("【Loop処理】複数国の災害分析")
print("="*60)
print(f"分析対象国: {', '.join(target_countries)}\n")

# 統計情報を保存するリスト
all_stats = []

# for文で各国を分析
for i, country in enumerate(target_countries, 1):
    print(f"\n{'='*60}")
    print(f"[{i}/{len(target_countries)}] 分析中: {country}")
    print(f"{'='*60}")
    
    # 関数を呼び出して分析
    stats = analyze_disaster_by_country(country, start_year=1970, end_year=2024)
    
    if stats:
        all_stats.append(stats)
        print(f"\n✓ {country}の分析完了")

print(f"\n{'='*60}")
print(f"全{len(target_countries)}カ国の分析が完了しました")
print(f"{'='*60}")

## 4. 国別比較: 統計サマリー

for文で収集したデータを使って、国別の比較分析を行います。

In [ ]:
# 比較用データフレームの作成
if all_stats:
    comparison_df = pd.DataFrame(all_stats)
    comparison_df = comparison_df.sort_values('total_disasters', ascending=False)
    
    print("\n" + "="*80)
    print("国別災害統計サマリー (1970-2024)")
    print("="*80)
    print(comparison_df[['country', 'total_disasters', 'total_deaths', 'total_affected', 'most_common_disaster']].to_string(index=False))
    
    # 比較グラフの作成
    fig, axes = plt.subplots(1, 2, figsize=(15, 5))
    fig.suptitle('国別災害統計比較 (1970-2024)', fontsize=14, fontweight='bold')
    
    # 総災害件数の比較
    axes[0].barh(comparison_df['country'], comparison_df['total_disasters'], color='coral')
    axes[0].set_xlabel('総災害件数')
    axes[0].set_title('国別総災害件数', fontweight='bold')
    axes[0].grid(True, alpha=0.3)
    
    # 総死者数の比較（対数スケール）
    axes[1].barh(comparison_df['country'], comparison_df['total_deaths'], color='steelblue')
    axes[1].set_xlabel('総死者数（人）')
    axes[1].set_title('国別総死者数', fontweight='bold')
    axes[1].set_xscale('log')
    axes[1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
else:
    print("統計データが収集されていません。")

## Week10課題まとめ

### 実装された機能

✅ **自作Function**: `analyze_disaster_by_country()` - 国別災害分析関数  
✅ **サンプル出力**: 日本の災害データを詳細に分析  
✅ **Loop処理**: for文による5カ国の一括分析  

### 分析結果から得られた知見

1. **災害発生パターンの地域差**
   - アジア太平洋地域は、地震、台風、洪水などの自然災害が頻発
   - 各国で最多災害タイプが異なり、地理的特性を反映

2. **災害件数の経年変化**
   - 1970年代以降、報告件数が全体的に増加傾向
   - 気候変動の影響や、報告システムの改善が要因と考えられる

3. **人的被害の規模**
   - 国によって総死者数に大きな差がある
   - 防災インフラや早期警報システムの整備状況が影響

---

**Week10課題完了**  
**データソース**: EM-DAT (Emergency Events Database)
